In [2]:
import os
from langchain_openai import AzureOpenAIEmbeddings

os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-03-01-preview"
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
)

from langchain.storage import LocalFileStore, RedisStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import FAISS
store = RedisStore(redis_url="redis://localhost:6379")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
embeddings, store, namespace="openai"
)
row_string = []
with open('./data/clean_relations', 'r') as f:
    data = [line.strip() for line in f]
db = FAISS.from_texts(data, cached_embedder)
retriever = db.as_retriever(search_kwargs={"k": 5})

### Create with GPT


In [6]:
from langchain_openai import AzureChatOpenAI
import os

os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=1,
    n = 3,
    max_retries=5, request_timeout=600
)
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain

In [ ]:
# The name of Justin Bieber's brother is Jaxon Bieber. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "Who was the prime minister of Japan in 2011, that served in the New Party Sakigake?",
            "evidence": "Relations retrieved: language.human_language.countries_spoken_in\n Entities retrieved: Japan",
            "preceding_sentences": "",
            "output": "Naoto Kan",
            "rating": "[Continue to Retrieve Evidence]"
        }],
        example_prompt=PromptTemplate.from_template("""Query: {query}
Preceding sentences: {preceding_sentences}
Evidence: {evidence}
Output: {target_output}
Rating: {rating}"""),
        prefix=
        "You will be provided with a query, evidence, output sentence, and preceding sentences (optional). Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. There are three cases:\n" 
        "- If the output sentence can be verified solely with the evidence and the preceding sentences, then respond with [No Retrieval]. \n"
        "- If additional information about the tail entity in evidence is needed to verify the output sentence, respond with [Continue to Retrieve Evidence] \n"
        "- If more information unrelated to the evidence is needed, e.g. totally new relationship or new entity, reponse with [New Retrieval].\n\n",
        suffix=
        "Query: {query}\n"
        "Preceding sentences: {preceding_sentences}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}",
        input_variables=["query", "evidence", "preceding_sentences", "topic"],
)

llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


In [ ]:
# relationship relevance
graph_intepretable =  """You will receive a query, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out valid information from the evidence to answer the given query, evaluate your output and provide explanations on your result.

###
Query: Name the president of the country whose main spoken language was Brahui in 1980?
Topic Entity: Brahui Language
Evidence: language.human_language.main_country; language.human_language.language_family; language.human_language.iso_639_3_code; base.rosetta.languoid.parent; language.human_language.writing_system; base.rosetta.languoid.languoid_class; language.human_language.countries_spoken_in; kg.object_profile.prominent_type; base.rosetta.languoid.document; base.ontologies.ontology_instance.equivalent_instances; base.rosetta.languoid.local_name; language.human_language.region
Preceding sentences: 
Output: 
1. {{language.human_language.main_country (Score: Fully relavant))}}: This relation is highly relevant as it directly relates to the country whose president is being asked for, and the main country where Brahui language is spoken in 1980.
2. {{language.human_language.countries_spoken_in (Score: Fully relavant)}}: This relation is also relevant as it provides information on the countries where Brahui language is spoken, which could help narrow down the search for the president.
3. {{base.rosetta.languoid.parent (Score: Partially relevant)}}: This relation is less relevant but still provides some context on the language family to which Brahui belongs, which could be useful in understanding the linguistic and cultural background of the country in question.

###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: 
"""
# The name of Justin Bieber's brother is Jaxon Bieber. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "Name the president of the country whose main spoken language was Brahui in 1980?",
            "topic": "Brahui Language",
            "evidence": "language.human_language.main_country; language.human_language.language_family; language.human_language.iso_639_3_code; base.rosetta.languoid.parent; language.human_language.writing_system; base.rosetta.languoid.languoid_class; language.human_language.countries_spoken_in; kg.object_profile.prominent_type; base.rosetta.languoid.document; base.ontologies.ontology_instance.equivalent_instances; base.rosetta.languoid.local_name; language.human_language.region",
            "preceding_sentences": "",
            "output": """1. {{language.human_language.main_country (Score: [Fully Relavant])}}: This relation is highly relevant as it directly relates to the country whose president is being asked for, and the main country where Brahui language is spoken in 1980.
2. {{language.human_language.countries_spoken_in (Score: [Fully Relavant])}}: This relation is also relevant as it provides information on the countries where Brahui language is spoken, which could help narrow down the search for the president.
3. {{base.rosetta.languoid.parent (Score: [Partially Relevant])}}: This relation is less relevant but still provides some context on the language family to which Brahui belongs, which could be useful in understanding the linguistic and cultural background of the country in question."""
        }],
        example_prompt=PromptTemplate.from_template("""###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: {output}
"""),
        prefix=
        """You will receive a query, topic entity, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out 3 valid information from the evidence that contribute to answering the query and provide a relevance score for each output, output your explanations for the score.
The score of relevance range from [Fully Relavant], [Partially Relevant] to [Unrelevant].""",
        suffix=
        """###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: """,
        input_variables=["query", "evidence", "preceding_sentences", "topic"],
)
graph_intepretable_prompt = PromptTemplate(input_variables=["query", "evidence", "preceding_sentences", "topic"], template=
graph_intepretable)
llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "what is the name of justin bieber brother?",
            "output": "[Retreive New Relationship]<paragraph>people.sibling_relationship.sibling;fictional_universe.fictional_character.siblings;fictional_universe.sibling_relationship_of_fictional_characters.siblings;people.person.sibling_s;people.family.members;people.person.parents</paragraph>Retrieved relationship: people.person.parents[Fully Relevant][Retrieve Entity]<paragraph>(Justin Bieber, people.person.parents, Pattie Mallette);(Justin Bieber, people.person.parents, Jeremy Bieber);(Justin Bieber, people.person.parents, Jeremy Bieber)</paragraph>Retrieved triplet: (Justin Bieber, people.person.parents, Jeremy Bieber)[Fully Relevant][Continue to Retrieve Evidence]<paragraph>people.sibling_relationship.sibling;people.person.sibling_s;people.person.parents;fictional_universe.fictional_character.siblings;people.family.members;people.person.children</paragraph>Retrieved relationship: people.person.children[Fully Relevant][Retrieve Entity]<paragraph>(Jeremy Bieber, people.person.children, Jazmyn Bieber);(Jeremy Bieber, people.person.children, Justin Bieber);(Jeremy Bieber, people.person.children, Jaxon Bieber);(Jeremy Bieber, people.person.children, Jaxon Bieber)</paragraph>Retrieved triplet: (Jeremy Bieber, people.person.children, Jaxon Bieber)[Fully Relevant][No Retrieval] Answer: Jaxon Bieber",
            "explanation": "The output provides the name of justin bieber's brother. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.",
            "rating": "[Confidence:5]"
        }],
        example_prompt=PromptTemplate.from_template("""
Query: {query}\n
Output: {output}
Explanation: {explanation}
Rating: {rating}
"""),
        prefix="""Given a query and an output, rate whether the response and the thought process appears to be a helpful and informative answer to the query, from 1 (lowest) - 5 (highest). We call this confidence score.
[Confidence:5]: The response provides a complete and correct reasoning chain to the query, and the final answer is complete and logically correct.
[Confidence:4]: The response mostly fulfills the need in the query and provides correct answers, while there can be some minor improvements such as shorter reasoning chain or less repetition.
[Confidence:3]: The response is acceptable, but the answers may be not complete or needs minor improvement.
[Confidence:2]: The reasoning process still addresses the main request, but the answers are not correct or not relevant to the query.
[Confidence:1]: The reasoning is barely irrelevant or does not give an answer in the end.""",
        suffix=
        """
Query: {query}\n
Output: {output}\n""",
        input_variables=["query", "output"],
)
# confidence_prompt = PromptTemplate(input_variables=["query", "output"], template=
# graph_intepretable)
llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


### Run_long_form answer

In [3]:
from vllm import LLM, SamplingParams
model = LLM(model='/media/disk2/llama_factory/generation_1124_special', trust_remote_code=True)



/media/disk1/chatgpt/miniconda3/envs/self-rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WARNING 12-05 01:15:27 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.


2024-12-05 01:15:29,280	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-05 01:15:33 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/media/disk2/llama_factory/generation_1124_special', speculative_config=None, tokenizer='/media/disk2/llama_factory/generation_1124_special', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/media/disk2/llama_factory/generation_1124_special, num_scheduler_s

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:01<00:04,  1.41s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:07<00:08,  4.06s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:13<00:05,  5.05s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:20<00:00,  5.87s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:20<00:00,  5.17s/it]



INFO 12-05 01:15:55 model_runner.py:1067] Loading model weights took 14.9605 GB
INFO 12-05 01:15:56 gpu_executor.py:122] # GPU blocks: 9656, # CPU blocks: 2048
INFO 12-05 01:15:56 gpu_executor.py:126] Maximum concurrency for 8192 tokens per request: 18.86x
INFO 12-05 01:15:58 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-05 01:15:58 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 12-05 01:16:09 model_runner.py:1523] Graph capturing finished in 12 secs.


In [3]:
sampling_params = SamplingParams(
            temperature=0.5, top_p=0.8, max_tokens=2048, skip_special_tokens=False, include_stop_str_in_output=True)
model.generate('Hello! Who are you?', sampling_params,)


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.51it/s, est. speed input: 10.58 toks/s, output: 60.45 toks/s]


[RequestOutput(request_id=0, prompt='Hello! Who are you?', prompt_token_ids=[128000, 9906, 0, 10699, 527, 499, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=" What brings you to our neck of the woods? We're a friendly bunch, so don't be shy! Say hello and let us know what's on your mind. We're all ears!<|eot_id|>", token_ids=(3639, 12716, 499, 311, 1057, 13272, 315, 279, 33633, 30, 1226, 2351, 264, 11919, 15860, 11, 779, 1541, 956, 387, 33394, 0, 25961, 24748, 323, 1095, 603, 1440, 1148, 596, 389, 701, 4059, 13, 1226, 2351, 682, 25212, 0, 128009), cumulative_logprob=None, logprobs=None, finish_reason=stop, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1733130983.477379, last_token_time=1733130983.477379, first_scheduled_time=1733130983.4866183, first_token_time=1733130983.5325885, time_in_queue=0.00923919677734375, finished_time=1733130984.1349523, scheduler_time=0.0043992819264531136, mod

In [4]:
#是否带special token 分开传入，用于检索和生成
sampling_params = SamplingParams(
            temperature=0.5, top_p=1.0,max_tokens=100, logprobs=5, skip_special_tokens=False, include_stop_str_in_output=True)
PROMPT_DICT = {"llama3": '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'}
model.generate([PROMPT_DICT["llama3"].format(input="what kind of money to take to bahamas") + "finance.currency.countries_used[Fully Relevant]location.country.currency_used[Fully Relevant]finance.currency.currency_code[Partially Relevant][Retrieve Entity]Bahamian dollar[Fully Relevant]"], sampling_params)[0].outputs[0]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.59it/s, est. speed input: 106.52 toks/s, output: 51.96 toks/s]


CompletionOutput(index=0, text='finance.currency.countries_used[Partially Relevant]location.country.currency_used[Partially Relevant][Retrieve Entity][No Retrieval]Answer: Bahamian dollar<|eot_id|>', token_ids=(63775, 46518, 522, 20730, 28903, 128262, 2588, 34424, 46518, 28903, 128262, 128259, 128256, 16533, 25, 32429, 309, 1122, 18160, 128009), cumulative_logprob=-3.1009353531653687, logprobs=[{63775: Logprob(logprob=-3.0577995777130127, rank=2, decoded_token='finance'), 128256: Logprob(logprob=-0.057799555361270905, rank=1, decoded_token='[No Retrieval]'), 2588: Logprob(logprob=-4.807799339294434, rank=3, decoded_token='location'), 128258: Logprob(logprob=-6.932799339294434, rank=4, decoded_token='[Continue to Retrieve Evidence]'), 23175: Logprob(logprob=-11.307799339294434, rank=5, decoded_token='United')}, {46518: Logprob(logprob=0.0, rank=1, decoded_token='.currency'), 522: Logprob(logprob=-18.75, rank=2, decoded_token='.c'), 11667: Logprob(logprob=-21.75, rank=3, decoded_token=' 

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/media/disk2/llama_factory/generation_1124_special')

In [6]:
def load_special_tokens(tokenizer, use_grounding=False, use_utility=False):
    rel_tokens = {}
    for token in ['[Unrelevant]','[Partially Relevant]','[Fully Relevant]']:
        rel_tokens[token] = tokenizer.convert_tokens_to_ids(token)

    # ut_tokens = None
    # if use_utility is True:
    #     ut_tokens = {}
    #     for token in utility_tokens_names:
    #         ut_tokens[token] = tokenizer.convert_tokens_to_ids(token)

    return rel_tokens

In [7]:
rel_tokens = load_special_tokens(tokenizer)

In [8]:
import numpy as np
import re
from src.sparql_utils import *
def run_step_generation_batch(model, prompt, topic_entity,new_retrieval, beam_width=3):
    pattern = r'(.*?)\[(.*?)\]'
    rel_score_dict = {}
    return_entities = []
    final_preds = []
    overall_scores = {}
    paragraph = ';'.join([page.page_content.strip() for page in retriever.invoke(prompt.split('\n\n')[1].split('<|eot_id|>')[0])])
    print(paragraph)
    if new_retrieval:
        retrieval_token = "[New Retrieval]"
        aug_prompts =  ["<paragraph>{}</paragraph>".format(paragraph)]
    else:
        retrieval_token = "[Continue to Retrieve Evidence]"
        aug_prompts =  ["<paragraph>{}</paragraph>".format(paragraph)]
    
    pred = model.generate(prompt + retrieval_token + aug_prompts[0], sampling_params)[0]
    pred_token_ids = pred.outputs[0].token_ids
    pred_text_1 = pred.outputs[0].text
    pred_log_probs = pred.outputs[0].logprobs
    seq_score = pred.outputs[0].cumulative_logprob / \
        max(len(pred.outputs[0].token_ids), 1)
    relevance_indices = []
    for tok_idx, tok in enumerate(pred_token_ids):
        if tok in rel_tokens.values():
            relevance_indices.append(tok_idx)
    if len(relevance_indices) > 0:
        for idx in relevance_indices:
            for token, token_id in rel_tokens.items():
                prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
                rel_score_dict[token] = np.exp(prob)
    relevance_score = rel_score_dict['[Fully Relevant]']+ rel_score_dict['[Partially Relevant]'] / np.sum(list(rel_score_dict.values()))
    processed_pred = pred_text_1.split('[Retrieve Entity]')[0]
    matches =  dict(re.findall(pattern,processed_pred))
    
    name2id = dict()
    entity_prompts = []
    for _, entity in enumerate(topic_entity):
        entities = []
        for k, v in matches.items():
            if v in ['Fully Relevant', 'Partially Relevant']:
                another_entities = get_another_entity(entity, k, return_label=True)
                # print(another_entities)
                name2id.update(another_entities)
                entities.extend([f'({get_label(entity)}, {k}, {e})' for e in another_entities.values()])
        entity_prompts.append(aug_prompts[0] + processed_pred +  '[Retrieve Entity]' + "<paragraph>{}</paragraph>".format(';'.join(entities[:10])))
    # print(aug_prompts)
    preds = model.generate([prompt + retrieval_token+ entity_prompts[i] for i in range(len(entity_prompts))], sampling_params)
    
    for p_idx, pred in enumerate(preds):
        pred_token_ids = pred.outputs[0].token_ids
        pred_text_2 = pred.outputs[0].text
        pred_log_probs = pred.outputs[0].logprobs
        rel_score_dict = {}
        relevance_indices = []
        for tok_idx, tok in enumerate(pred_token_ids):
            if tok in rel_tokens.values():
                relevance_indices.append(tok_idx)
        if len(relevance_indices) > 0:
            # print(relevance_indices)
            for idx in relevance_indices:
                for token, token_id in rel_tokens.items():
                    prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
                    rel_score_dict[token] = np.exp(prob)
        overall_scores[p_idx] = relevance_score + rel_score_dict['[Fully Relevant]'] + rel_score_dict['[Partially Relevant]']/ np.sum(list(rel_score_dict.values()))
        if '[Continue to Retrieve Evidence]' in pred_text_2:
            processed_pred = pred_text_2.split('[Continue to Retrieve Evidence]')[0]
            matches =  dict(re.findall(pattern, processed_pred))
            for k, v in matches.items():
                if v in ['Fully Relevant', 'Partially Relevant']:
                    if k in name2id:
                        return_entities.append(name2id[k])
            processed_pred += '[Continue to Retrieve Evidence]'
        else:
            processed_pred = pred_text_2
        final_preds.append(aug_prompts[0] + processed_pred)
    return final_preds, [overall_scores[p_idx] for p_idx in overall_scores], return_entities

        

In [9]:
import json
with open('./data/merged/WebQSP_test.json', 'r') as f:
    test_data = json.load(f)
print(len(test_data))

1639


In [12]:
count = 0
# for i in range(len(test_data)):
i=4
print(f'Process {i}')
data_input = test_data[i]['question']
prompt = PROMPT_DICT['llama3'].format(input= data_input)
max_depth = 3
topic_entity = list(test_data[i]['gold_entity_map'].keys())
# pred = model.generate([prompt], sampling_params)[0]
# pred_text = pred.outputs[0].text
# if '[New Retrieval]' in pred_text:
curr_depth = 1
terminated = False
node_id = 0
prediction_tree = {}
levels = {}
prediction_tree[node_id] = {"prompt": prompt, "pred": "[New Retrieval]",
                            "processed_pred": "", "score": None, "topic_entity": topic_entity, "parent": None}
levels[0] = [0]
while curr_depth < max_depth:
    levels[curr_depth] = []
    if curr_depth-1 in levels and terminated is False:
        for node in levels[curr_depth-1]:
            pred = prediction_tree[node]["pred"]
            if "<|eot_id|>" in pred:
                terminated = True
                continue
            prompt = prediction_tree[node]["prompt"]
            prev_generation = prediction_tree[node]["processed_pred"]
            score = prediction_tree[node]["score"]
            topic_entity = prediction_tree[node]["topic_entity"]
            if "[New Retrieval]" in pred or "[Continue to Retrieve Evidence]" in pred:
                retrieval_results = {}
                preds, scores, next_entities = run_step_generation_batch(
                    model, prompt + prev_generation, topic_entity, new_retrieval=True if ("[New Retrieval]" in pred) else False)
                for i, (pred, p_score) in enumerate(zip(preds, scores)):
                    retrieval_results[i] = {
                        "pred": pred, "score": p_score}

                for i, result in retrieval_results.items():
                    node_id += 1
                    node_score = result["score"] * \
                        score if score is not None else result["score"]
                    pred = result["pred"]
                    if len(next_entities) == 0:
                        next_entities = [topic_entity]
                    prediction_tree[node_id] = {"prompt": prompt + prev_generation, "pred": pred,
                                                "score": node_score, "parent": node,
                                                "topic_entity": next_entities[0]}
                    if "[Continue to Retrieve Evidence]" in pred:
                        gen_result_index = pred.index("[Continue to Retrieve Evidence]")
                        prev_generation = pred[:gen_result_index]
                    else:
                        prev_generation = pred
                    prediction_tree[node_id]["processed_pred"] = prev_generation
                    levels[curr_depth].append(node_id)
            current_rank = levels[curr_depth]
            node2score = {
                node_id: prediction_tree[node_id]["score"] for node_id in current_rank}
            top_nodes = sorted(node2score.items(), key=lambda x: x[1], reverse=True)[
                :2]
            levels[curr_depth] = [node[0] for node in top_nodes]
            curr_depth += 1
    else:
        break
labels = [get_label(ans) for ans in test_data[i]['answer']]
for label in labels:
    if label in prediction_tree[len(prediction_tree)-1]['processed_pred']:
        print(f'correct {i}')
        count += 1



Process 4
location.us_state.capital;people.family.country;people.ethnicity.geographic_distribution;fictional_universe.fictional_character.place_of_birth;people.marriage.from


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.54it/s, est. speed input: 193.99 toks/s, output: 63.81 toks/s]


In [15]:
list(test_data[i]['gold_entity_map'].keys())

['m.03_r3']

In [16]:
prediction_tree

{0: {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nwhere was george washington carver from<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[New Retrieval]',
  'processed_pred': '',
  'score': None,
  'topic_entity': ['m.03djm'],
  'parent': None},
 1: {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nwhere was george washington carver from<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '<paragraph>location.us_state.capital;people.family.country;people.ethnicity.geographic_distribution;fictional_universe.fictional_character.place_of_birth;people.marriage.from</paragraph>people.family.country[Fully Relevant]people.ethnicity.geographic_distribution[Partially Relevant]location.us_state.capital[Unrelevant][Retrieve Entity]<paragraph>[No Retrieval]Answer: Minneapolis<|eot_id|>',
  'score': 2.578040019318527e-07,
  'parent': 0,
  'topic_entity': ['m.03djm'],
  'processed_pred': '<paragraph>location.us_sta

In [ ]:
def backtracking_prediction_tree(levels: dict[int,list[int]], curr_depth: int, prediction_tree: dict[int, dict]) -> dict[int,list[int]]:
    '''
    get best tracking from prediction_tree base on levels
    '''
    parent = 0 
    best_selections = {}
    # Traverse from the bottom 
    levels = {k: v for k, v in levels.items() if len(v) > 0 and k != 0} # remove empty list in levels
    for path_i, node in enumerate(levels[len(levels)]): # beam search 
        if node == 0:
            break
        best_selections[path_i] = [node] 
        current_node = node 
        current_level = curr_depth 
        if current_node is None:
            continue
        while current_level > 0 and current_node is not None:
            parent = prediction_tree[current_node]["parent"]
            best_selections[path_i] = [parent] + best_selections[path_i] 
            current_node = parent 
            current_level -= 1
    return best_selections


In [ ]:
best_selections = backtracking_prediction_tree(levels, curr_depth, prediction_tree)
final_prediction = {}
splitted_sentences = {}
original_splitted_sentences = {}
ctxs = {}
for path_i, nodes in best_selections.items():
    final_prediction[path_i] = " ".join([prediction_tree[node]["processed_pred"] for node in nodes if node is not None])

In [15]:
print(final_prediction)

{0: ' <paragraph>government.us_vice_president.to_president;government.us_president.vice_president;government.politician.government_positions_held;government.political_appointer.appointees;base.obamabase.cabinet_member.cabinet_position</paragraph>None[Fully Relevant]George M. Dallas[Unrelevant][No Retrieval]Answer: Speaker of the House of Representatives;President of the House of Representatives<|eot_id|>'}


In [ ]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
prompt = 'Hello!' 
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.completions.create(
    model="/media/disk1/chatgpt/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920",
    prompt=prompt,
    max_tokens=2048,
#     extra_body={
#     "skip_special_tokens": "True",
#   }
)
print("Chat response:", chat_response)
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8001/v1"
prompt = 'what country is the grand bahama island in?' 
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="/media/disk2/llama_factory/generation_1124_special/",
    messages=[
        {"role": "user", "content": prompt},
    ],
    extra_body={
    "skip_special_tokens": "False",
    #  "logprobs": "True",
  }
)
print("Chat response:", chat_response)

Chat response: Completion(id='cmpl-d7ccc07b74794709a9452c6f4f43865a', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text=' Thank you for purchasing this Kindle edition of The Way Life Should Be, written by Irma Coriss. Irma is a lifelong resident of South Florida and currently resides in St. Lucie County. This is her third novel and her first book in her RUMBUNCH series. The idea of this series came about as Irma wished to write stories that would make her audience believe in the goodness of life, having faith that all is not invisible, and just believing that with careful attention to detail, you, too, live as if life\'s moments are "The Way Life Should Be". Hope you enjoy Irma\'s stories and feel she has allowed all of us to believe that God\'s love goes and on and on and today belongs to you.\nIrma Coriss is a graduate of one of the first degrees offered to a woman. While attending college, she fell in love with writing and took some of her professors\' fav

### Tree fix

In [45]:
import numpy as np
import re
from src.sparql_utils import *
def run_relation_generation_batch(model, prompt, new_retrieval, context):
    rel_score_dict = {}
    final_preds = []
    overall_scores = {}
    paragraph = ';'.join([page.page_content.strip() for page in retriever.invoke(prompt.split('\n\n')[1].split('<|eot_id|>')[0])])
    if new_retrieval:
        retrieval_token = "[New Retrieval]"
        aug_prompts =  ["<paragraph>{}</paragraph>".format(paragraph)]
    else:
        retrieval_token = "[Continue to Retrieve Evidence]"
        aug_prompts =  ["<paragraph>{}</paragraph>".format(paragraph)]
    
    pred = model.generate(prompt + retrieval_token + aug_prompts[0], sampling_params)[0]
    pred_token_ids = pred.outputs[0].token_ids
    pred_text_1 = pred.outputs[0].text
    pred_log_probs = pred.outputs[0].logprobs
    seq_score = pred.outputs[0].cumulative_logprob / \
        max(len(pred.outputs[0].token_ids), 1)
    relevance_indices = []
    for tok_idx, tok in enumerate(pred_token_ids):
        if tok in rel_tokens.values():
            relevance_indices.append(tok_idx)
    if len(relevance_indices) > 0:
        for idx in relevance_indices:
            for token, token_id in rel_tokens.items():
                prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
                rel_score_dict[token] = np.exp(prob)
    relevance_score = rel_score_dict['[Fully Relevant]']+ rel_score_dict['[Partially Relevant]'] / np.sum(list(rel_score_dict.values()))
    assert '[Retrieve Entity]' in pred_text_1
    processed_pred = pred_text_1.split('[Retrieve Entity]')[0] + '[Retrieve Entity]'
    return [retrieval_token + aug_prompts[0] + processed_pred], [relevance_score], pred_text_1.split('[Retrieve Entity]')[0]

def run_entity_generation_batch(model, prompt, topic_entity, context):
    final_preds = []
    overall_scores = {}
    return_entities = []
    pattern = r'(.*?)\[(.*?)\]'
    matches =  dict(re.findall(pattern,context))
    name2id = dict()
    entity_prompts = []
    for _, entity in enumerate(topic_entity):
        entities = []
        for k, v in matches.items():
            if v in ['Fully Relevant', 'Partially Relevant']:
                another_entities = get_another_entity(entity, k, return_label=True)
                # print(another_entities)
                name2id.update(another_entities)
                entities.extend([f'({get_label(entity)}, {k}, {e})' for e in another_entities.values()])
        
        entity_prompts.append(prompt+  '[Retrieve Entity]' + "<paragraph>{}</paragraph>".format(';'.join(entities[:10]) if len(entities) else 'No triplets Received'))
    # print(aug_prompts)
    preds = model.generate([prompt + entity_prompts[i] for i in range(len(entity_prompts))], sampling_params)
    
    for p_idx, pred in enumerate(preds):
        pred_token_ids = pred.outputs[0].token_ids
        pred_text_2 = pred.outputs[0].text
        pred_log_probs = pred.outputs[0].logprobs
        rel_score_dict = {}
        relevance_indices = []
        for tok_idx, tok in enumerate(pred_token_ids):
            if tok in rel_tokens.values():
                relevance_indices.append(tok_idx)
        if len(relevance_indices) > 0:
            # print(relevance_indices)
            for idx in relevance_indices:
                for token, token_id in rel_tokens.items():
                    prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
                    rel_score_dict[token] = np.exp(prob)
        overall_scores[p_idx] = rel_score_dict['[Fully Relevant]'] + rel_score_dict['[Partially Relevant]']/ np.sum(list(rel_score_dict.values()))
        if '[Continue to Retrieve Evidence]' in pred_text_2:
            processed_pred = pred_text_2.split('[Continue to Retrieve Evidence]')[0]
            matches =  dict(re.findall(pattern, processed_pred))
            for k, v in matches.items():
                if v in ['Fully Relevant', 'Partially Relevant']:
                    if k in name2id:
                        return_entities.append(name2id[k])
            processed_pred += '[Continue to Retrieve Evidence]'
        else:
            processed_pred = pred_text_2
        final_preds.append(processed_pred)
    return final_preds, [overall_scores[p_idx] for p_idx in overall_scores], return_entities, 

        

In [ ]:
count = 0
for index in range(len(test_data)):
# index=7
    print(f'Process {index}')
    data_input = test_data[index]['question']
    prompt = PROMPT_DICT['llama3'].format(input= data_input)
    max_depth = 5
    topic_entity = list(test_data[index]['gold_entity_map'].keys())
    # pred = model.generate([prompt], sampling_params)[0]
    # pred_text = pred.outputs[0].text
    # if '[New Retrieval]' in pred_text:
    curr_depth = 1
    terminated = False
    node_id = 0
    prediction_tree = {}
    levels = {}
    prediction_tree[node_id] = {"prompt": prompt, "pred": "[New Retrieval]",
                                "processed_pred": "", "score": None, "topic_entity": topic_entity, "parent": None, "context": None}
    levels[0] = [0]
    while curr_depth < max_depth:
        levels[curr_depth] = []
        if curr_depth-1 in levels and terminated is False:
            for node in levels[curr_depth-1]:
                curr_pred = prediction_tree[node]["pred"]
                if "<|eot_id|>" in curr_pred:
                    terminated = True
                    continue
                prompt = prediction_tree[node]["prompt"]
                prev_generation = prediction_tree[node]["processed_pred"]
                score = prediction_tree[node]["score"]
                topic_entity = prediction_tree[node]["topic_entity"]
                context = prediction_tree[node]['context']
                if "Retrieve Entity" in curr_pred.split('[')[-1]:
                    retrieval_results = {}
                    preds, scores, next_entities = run_entity_generation_batch(
                        model, prompt + prev_generation, topic_entity, context)
                    for i, (pred, p_score) in enumerate(zip(preds, scores)):
                        retrieval_results[i] = {
                            "pred": pred, "score": p_score}

                    for i, result in retrieval_results.items():
                        node_id += 1
                        node_score = result["score"] * \
                            score if score is not None else result["score"]
                        pred = result["pred"]
                        if len(next_entities) == 0:
                            next_entities = topic_entity
                        prediction_tree[node_id] = {"prompt": prompt + prev_generation, "pred": pred, "context": next_entities,
                                                    "score": node_score, "parent": node,
                                                    "topic_entity": next_entities}
                        if "[Continue to Retrieve Evidence]" in pred:
                            gen_result_index = pred.index("[Continue to Retrieve Evidence]")
                            prev_generation = pred[:gen_result_index]
                        else:
                            prev_generation = pred
                        prediction_tree[node_id]["processed_pred"] = prev_generation
                        levels[curr_depth].append(node_id)
                #存在前后逻辑粘连   
                if "New Retrieval" in curr_pred.split('[')[-1] or "Continue to Retrieve Evidence" in curr_pred.split('[')[-1]:
                    retrieval_results = {}
                    preds, scores, context = run_relation_generation_batch(
                        model, prompt + prev_generation, new_retrieval=True if ("[New Retrieval]" in curr_pred) else False, context=context)
                    for i, (pred, p_score) in enumerate(zip(preds, scores)):
                        retrieval_results[i] = {
                            "pred": pred, "score": p_score}

                    for i, result in retrieval_results.items():
                        node_id += 1
                        node_score = result["score"] * \
                            score if score is not None else result["score"]
                        pred = result["pred"]
                        prediction_tree[node_id] = {"prompt": prompt + prev_generation, "pred": pred,
                                                    "score": node_score, "parent": node,
                                                    "topic_entity": topic_entity, "context": context}
                        if "[Retrieve Entity]" in pred:
                            gen_result_index = pred.index("[Retrieve Entity]")
                            prev_generation = pred[:gen_result_index]
                        else:
                            prev_generation = pred
                        prediction_tree[node_id]["processed_pred"] = prev_generation
                        levels[curr_depth].append(node_id)
            current_rank = levels[curr_depth]
            node2score = {
                node_id: prediction_tree[node_id]["score"] for node_id in current_rank}
            top_nodes = sorted(node2score.items(), key=lambda x: x[1], reverse=True)[
                :2]
            levels[curr_depth] = [node[0] for node in top_nodes]
            curr_depth += 1
        else:
            break
    labels = [get_label(ans) for ans in test_data[index]['answer']]
    for label in labels:
        if label in prediction_tree[len(prediction_tree)-1]['processed_pred']:
            count += 1
            break



Process 7


Processed prompts: 100%|██████████| 2/2 [00:01<00:00,  1.26it/s, est. speed input: 552.91 toks/s, output: 125.95 toks/s]


In [34]:
levels

{0: [0], 1: [1], 2: [2, 4], 3: []}

In [50]:
prediction_tree

{0: {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nwho is governor of ohio 2011<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[New Retrieval]',
  'processed_pred': '',
  'score': None,
  'topic_entity': ['m.05kkh', 'm.0fkvn'],
  'parent': None,
  'context': None},
 1: {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nwho is governor of ohio 2011<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[New Retrieval]<paragraph>government.us_vice_president.to_president;government.us_president.vice_president;government.government_position_held.district_represented;government.election.winner;government.politician.election_campaigns</paragraph>government.election.winner[Fully Relevant]government.government_position_held.district_represented[Partially Relevant]government.politician.election_campaigns[Partially Relevant][Retrieve Entity]',
  'score': 0.999987954659157,
  'parent': 0,
  'topic_entity': ['m.05kkh